<a href="https://colab.research.google.com/github/ppiont/tensor-flow-state/blob/master/Model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/gdrive", force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
%cd "/gdrive/My Drive/tensor-flow-state/tensor-flow-state"

/gdrive/My Drive/tensor-flow-state/tensor-flow-state


In [0]:
import pandas as pd
df = pd.read_csv("data/df_imputed_week_shift.csv", index_col = 0, parse_dates = True)

In [0]:
# 0 resample
# 1 log transformere. 
# 2 difference ift. speed limit (som ogsaa skal vaere log transformeret)
# 3 

In [0]:
cols = ["speed", "flow", "speed_-2", "speed_-1","speed_+1", "speed_+2", "flow_-2", "flow_-1", "flow_+1", "flow_+2", "speed_limit"]
speed_cols = ["speed", "speed_-2", "speed_-1","speed_+1", "speed_+2"]
flow_cols = ["flow", "flow_-2", "flow_-1", "flow_+1", "flow_+2"]

### First resample to elmininate some noise

In [0]:
import scipy.stats as s
import numpy as np
def resample_df(df, freq = "10T", method_speed = np.median, method_flow = np.sum):
    copied = df.copy()
    copied = copied.resample(freq).agg({
           "speed": method_speed, "speed_-2": method_speed, "speed_-1": method_speed, "speed_+1": method_speed, "speed_+2": method_speed,
           "flow": method_flow, "flow_-2": method_flow, "flow_-1": method_flow, "flow_+1": method_flow, "flow_+2": method_flow,
           "speed_limit": np.median})
    return copied

In [0]:
r_df = resample_df(df, freq = "10T")

### Train test split. The final year is reserved for testing. (Val will be relevant later?)

In [0]:
def train_test_split(df, test_year):
    # train, test
    return df[df.index.year != test_year], df[df.index.year == test_year]

train, test = train_test_split(r_df, 2019)

In [46]:
df.iloc[:, -1]

2011-01-03 00:00:00    120.0
2011-01-03 00:01:00    120.0
2011-01-03 00:02:00    120.0
2011-01-03 00:03:00    120.0
2011-01-03 00:04:00    120.0
                       ...  
2019-12-29 23:55:00    120.0
2019-12-29 23:56:00    120.0
2019-12-29 23:57:00    120.0
2019-12-29 23:58:00    120.0
2019-12-29 23:59:00    120.0
Name: speed_limit, Length: 4727520, dtype: float64

In [0]:
import numpy as np
def log_transform(df):
    copy = df.copy()
    return np.log(copy.iloc[:, :-1].replace(0, 1e-15)).join(df.iloc[:, -1], how = 'inner')

# Log transform. First set 0s to very low value 'cause you can't log 0.
train_log = log_transform(train)

In [0]:
def calc_mean(df, col):
    # mean(100), mean(120)
    return df.groupby(['speed_limit'])[col].mean().values

def calc_sd(df, col):
    # sd(100), sd(120)
    return df.groupby(['speed_limit'])[col].std().values

def normalize_df(df, cols)
    copy = df.copy()
    for col in cols:
        # Find mean and sd for training batch
        mean100, mean120 = calc_mean(copy, col)
        sd100, sd1200 = calc_sd(copy, col)
        copy[col] = np.where(copy.speed_limit == 100, (copy[col] - mean100) / sd100, (copy[col] - mean120) / sd120)

In [42]:
train_norm = normalize_df()

array([ 97.76744005, 105.30305292])

In [36]:
mean100, mean120 = train.groupby(['speed_limit']).speed.mean().values
sd100, sd120 = train.groupby(['speed_limit']).std().values
print(mean100)
print(mean120)

97.76744004714918
105.30305292148017


In [9]:
# Find mean and sd for training batch
mean100, mean120 = train.groupby(['speed_limit']).mean().unstack().values
sd100, sd120_10m = train.groupby(['speed_limit']).std().unstack().values

# Normalize speed on training batch mean/sd (10m)
df_10m['speed_normalized'] = np.where(df_10m.speed_limit == 100, (df_10m.speed - mean100_10m) / sd100_10m, (df_10m.speed - mean120_10m) / sd120_10m)

# Plot Normalized distribution of training batch
df_10m.iloc[: -(31 * 24 * 6), df_10m.columns.get_loc('speed_normalized')].hist(bins = int(df_10m.speed.max() + 1))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4727520 entries, 2011-01-03 00:00:00 to 2019-12-29 23:59:00
Data columns (total 11 columns):
speed          float64
speed_-2       float64
speed_-1       float64
speed_+1       float64
speed_+2       float64
flow           float64
flow_-2        float64
flow_-1        float64
flow_+1        float64
flow_+2        float64
speed_limit    float64
dtypes: float64(11)
memory usage: 432.8 MB
